In [1]:
import os
import sys

src_dir = os.path.abspath("../src")
if src_dir not in sys.path:
    sys.path.append(src_dir)

In [3]:
# import the needed packages
import matplotlib.pyplot as plt
import numpy as np
import tools
import importlib
importlib.reload(tools)

import solver
import importlib
importlib.reload(solver)

from evolution_chained2_kicked import evolution_chained2_kicked
from tools import mv

from evolution_chained2_kicked import evolution_chained2_kicked
from tools import mv

In [2]:
# create bath

# chain size
ns = 30
# on-site energies
e = [1.0]*ns
# hoppings between sites
h = [0.05]*(ns-1)

In [3]:
# discretize the bath: apply the lightcone method
bth = solver.bath.from_chain(chain = (e, h), rel_tol = 10**(-7), ring_size = 5, t_max = 100, visualize = True)
bth.n_rel

8

In [4]:
bth.I_rel

array([ 3.82226204e+01,  3.24946857e+01,  2.09438833e+01,  7.19398453e+00,
        1.06964500e+00,  8.00242914e-02,  3.75964630e-03,  1.25098405e-04,
        3.13558596e-06,  6.15656931e-08,  9.74154822e-10,  1.26882635e-11,
        1.32461077e-13,  3.29432670e-17,  1.74769104e-18,  9.42510642e-19,
        2.71976630e-22,  2.23417943e-22,  3.69259838e-24,  6.83398577e-26,
        1.91917689e-26,  6.13387106e-29, -2.61725826e-36, -1.60675773e-34,
       -3.63998151e-32, -2.40156106e-31, -4.76995717e-31, -9.61643610e-20,
       -7.57367195e-17, -1.86143748e-15])

In [5]:
bth.I_rel / bth.I_rel[0]

array([ 1.00000000e+00,  8.50142804e-01,  5.47944727e-01,  1.88212751e-01,
        2.79846068e-02,  2.09363697e-03,  9.83618147e-05,  3.27288928e-06,
        8.20348245e-08,  1.61071356e-09,  2.54863432e-11,  3.31956925e-13,
        3.46551533e-15,  8.61878822e-19,  4.57239986e-20,  2.46584518e-20,
        7.11559353e-24,  5.84517600e-24,  9.66076723e-26,  1.78794277e-27,
        5.02105003e-28,  1.60477513e-30, -6.84740667e-38, -4.20368282e-36,
       -9.52310823e-34, -6.28308848e-33, -1.24794091e-32, -2.51590184e-21,
       -1.98146330e-18, -4.86998918e-17])

In [6]:
bth.n_rel

8

In [18]:
bth.itimes_in

[0, 1, 104, 642, 1710, 3191, 4943, 6869, 8865]

In [29]:
bth.m_in(642)

4

In [7]:
num_modes = 30
max_num_quanta = 4
m = tools.spin_boson_model(num_modes, max_num_quanta)

wq = 1.0
Hs = wq * m.s_p @ m.s_m
g = 0.1
V = g * m.s_m
V_dag = g * m.s_p

def f(ti):
    return(0.1*np.cos((ti + 0.5)*bth.dt))

psi = np.zeros(m.dimension, dtype = complex)
psi_mid = np.zeros(m.dimension, dtype = complex)
psi_mid_next = np.zeros(m.dimension, dtype = complex)
psi_buff = np.zeros(m.dimension, dtype = complex)

psi_ini = np.zeros(m.dimension, dtype = complex)
psi_ini[0] = 1
psi_begin = np.copy(psi_ini)

Ht = None

def begin_step(ti, psi):
    Hint = V_dag @ sum(bth.couplings_min[:, ti] * m.a_dag)
    Hint = Hint + Hint.conj().transpose()
    global Ht
    Ht = Hs + m.s_x * f(ti) + Hint
    
def apply_H(ti, psi_in, psi_out):
    mv(Ht, psi_in, psi_out)

def eval_o(ti, psi):
    pass

first_in_chain = 1
evolution_chained2_kicked(0, bth.nt, bth.dt, begin_step, apply_H, eval_o, psi_begin, psi, psi_mid, psi_mid_next, first_in_chain)

(array([1.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]),
 array([ 3.23077310e-01-1.57459858e-02j, -1.73464499e-11+9.76299102e-12j,
         6.91274430e-19+1.72273795e-20j, ...,
         3.04652880e-02-5.09928345e-02j, -1.80480297e-02+2.97865214e-02j,
         3.79016592e-03-6.15774066e-03j]),
 array([ 3.22949835e-01-1.57542351e-02j, -1.71368747e-11+9.78604414e-12j,
         6.92171324e-19+1.72933232e-20j, ...,
         3.07198353e-02-5.08416710e-02j, -1.81972876e-02+2.97107191e-02j,
         3.82113820e-03-6.14726876e-03j]),
 array([ 3.22949835e-01-1.57542351e-02j, -1.71368747e-11+9.78604414e-12j,
         6.92171324e-19+1.72933232e-20j, ...,
         3.07198353e-02-5.08416710e-02j, -1.81972876e-02+2.97107191e-02j,
         3.82113820e-03-6.14726876e-03j]))

In [ ]:
Nq = sum([m.a_dag[i] @ m.a[i] for i in range(20)])

In [9]:
psi_full = np.copy(psi) / np.sqrt(np.vdot(psi, psi))

In [10]:
np.vdot(psi_full, Nq @ psi_full)

(2.399123132040529+2.710505431213761e-20j)

In [11]:
num_modes = 30
max_num_quanta = 4
m = tools.spin_boson_model(num_modes, max_num_quanta)

#n_rel = 11

wq = 1.0
Hs = wq * m.s_p @ m.s_m
g = 0.1
V = g * m.s_m
V_dag = g * m.s_p

def f(ti):
    return(0.1*np.cos((ti + 0.5)*bth.dt))

psi = np.zeros(m.dimension, dtype = complex)
psi_mid = np.zeros(m.dimension, dtype = complex)
psi_mid_next = np.zeros(m.dimension, dtype = complex)
psi_buff = np.zeros(m.dimension, dtype = complex)

psi_ini = np.zeros(m.dimension, dtype = complex)
psi_ini[0] = 1
psi_begin = np.copy(psi_ini)

Ht = None

def begin_step(ti, psi):
    m_in = bth.m_in(ti)
    Hint = V_dag @ sum(bth.couplings_min[: m_in, ti] * m.a_dag[: m_in])
    Hint = Hint + Hint.conj().transpose()
    global Ht
    Ht = Hs + m.s_x * f(ti) + Hint
    
def apply_H(ti, psi_in, psi_out):
    mv(Ht, psi_in, psi_out)

def eval_o(ti, psi):
    pass

first_in_chain = 1
evolution_chained2_kicked(0, bth.nt, bth.dt, begin_step, apply_H, eval_o, psi_begin, psi, psi_mid, psi_mid_next, first_in_chain)

(array([1.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]),
 array([ 3.23083291e-01-1.57462004e-02j,  8.70456074e-11+5.46930326e-12j,
        -1.60994032e-20+3.73622954e-22j, ...,
         3.04653345e-02-5.09928699e-02j, -1.80479907e-02+2.97864877e-02j,
         3.79014033e-03-6.15771176e-03j]),
 array([ 3.22955812e-01-1.57544501e-02j,  8.69596502e-11+5.40267478e-12j,
        -1.60785943e-20+3.84057001e-22j, ...,
         3.07198820e-02-5.08417061e-02j, -1.81972484e-02+2.97106855e-02j,
         3.82111247e-03-6.14723998e-03j]),
 array([ 3.22955812e-01-1.57544501e-02j,  8.69596502e-11+5.40267478e-12j,
        -1.60785943e-20+3.84057001e-22j, ...,
         3.07198820e-02-5.08417061e-02j, -1.81972484e-02+2.97106855e-02j,
         3.82111247e-03-6.14723998e-03j]))

In [12]:
psi_fm = np.copy(psi) / np.sqrt(np.vdot(psi, psi))

In [13]:
np.vdot(psi_fm, Nq @ psi_fm)

(2.399109985208621+0j)

In [14]:
f = 1 - np.abs(np.vdot(psi_fm, psi_full))
f

2.4496275674579238e-08

In [15]:
np.vdot(psi_full, Nq @ psi_full)

(2.399123132040529+2.710505431213761e-20j)